In [1]:
import numpy as np 
import pandas as pd 
import csv
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import time

In [2]:
device = torch.device("mps")

In [3]:
# Geting training/validating dataset
x_train = []
x_label = []
val_train = []
val_label = []
raw_train = np.genfromtxt('train.csv', delimiter=',', dtype=str, skip_header=1)
for i in range(len(raw_train)):
    image = np.array(raw_train[i, 1].split(' ')).reshape(1, 48, 48)
    if (i % 10 == 0):
        val_train.append(image)
        val_label.append(raw_train[i][0])
    else:
        x_train.append(image)
        x_train.append(np.flip(image, axis=2))    # simple example of data augmentation
        x_label.append(raw_train[i][0])
        x_label.append(raw_train[i][0])
x_train = np.array(x_train, dtype=float) / 255.0
val_train = np.array(val_train, dtype=float) / 255.0
x_label = np.array(x_label, dtype=int)
val_label = np.array(val_label, dtype=int)

x_train = torch.FloatTensor(x_train)
val_train = torch.FloatTensor(val_train)
x_label = torch.LongTensor(x_label)
val_label = torch.LongTensor(val_label)
print(x_train.size(), val_train.size())

torch.Size([51676, 1, 48, 48]) torch.Size([2871, 1, 48, 48])


In [5]:
# Wrapped as dataloaders
train_set = TensorDataset(x_train, x_label)
val_set = TensorDataset(val_train, val_label)
batch_size = 256
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=8)

In [4]:
def gaussian_weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 and classname.find('Conv') == 0:
        m.weight.data.normal_(0.0, 0.02)

In [6]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1),  # [64, 24, 24]
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2, 0),      # [64, 12, 12]

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2, 0),      # [128, 6, 6]

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2, 0)       # [256, 3, 3]
        )

        self.fc = nn.Sequential(
            nn.Linear(256*3*3, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(p=0.5),
            nn.Linear(512, 7)
        )

        self.cnn.apply(gaussian_weights_init)
        self.fc.apply(gaussian_weights_init)

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


In [7]:
model = Classifier().to(device)
print(model)

Classifier(
  (cnn): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.2)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.2)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [12]:
# Set hyperparameters
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_acc = 0.0
num_epoch = 30

In [13]:
# Training
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        train_pred = model(data[0].to(device))
        batch_loss = loss(train_pred, data[1].to(device))
        batch_loss.backward()
        optimizer.step()
        
        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
        
        #Plot the progress
        progress = ('#' * int(float(i)/len(train_loader)*40)).ljust(40)
        print ('[%03d/%03d] %2.2f sec(s) | %s |' % (epoch+1, num_epoch, \
                (time.time() - epoch_start_time), progress), end='\r', flush=True)
        
    model.eval()
    for i, data in enumerate(val_loader):
        val_pred = model(data[0].to(device))
        batch_loss = loss(val_pred, data[1].to(device))

        val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        val_loss += batch_loss.item()

        progress = ('#' * int(float(i)/len(val_loader)*40)).ljust(40)
        print ('[%03d/%03d] %2.2f sec(s) | %s |' % (epoch+1, num_epoch, \
                (time.time() - epoch_start_time), progress), end='\r', flush=True)
        
    val_acc = val_acc/val_set.__len__()
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
        (epoch + 1, num_epoch, time.time()-epoch_start_time, \
        train_acc/train_set.__len__(), train_loss, val_acc, val_loss))
    
    if (val_acc > best_acc):
        with open('save/acc.txt','w') as f:
            f.write(str(epoch)+'\t'+str(val_acc)+'\n')
        torch.save(model.state_dict(), 'save/model.pth')
        best_acc = val_acc
        print ('Model Saved!')

print("Finish Training!")    

[001/030] 52.48 sec(s) Train Acc: 0.213774 Loss: 903.151809 | Val Acc: 0.269244 loss: 21.671302
Model Saved!
[002/030] 38.90 sec(s) Train Acc: 0.250774 Loss: 369.241852 | Val Acc: 0.284570 loss: 21.136398
Model Saved!
[003/030] 27.23 sec(s) Train Acc: 0.278427 Loss: 359.111871 | Val Acc: 0.308255 loss: 20.620061
Model Saved!
[004/030] 27.21 sec(s) Train Acc: 0.337778 Loss: 338.321874 | Val Acc: 0.370951 loss: 18.922485
Model Saved!
[005/030] 27.64 sec(s) Train Acc: 0.402044 Loss: 311.315560 | Val Acc: 0.428770 loss: 17.290852
Model Saved!
[006/030] 27.89 sec(s) Train Acc: 0.449319 Loss: 288.798981 | Val Acc: 0.386973 loss: 18.818995
[007/030] 27.73 sec(s) Train Acc: 0.485680 Loss: 270.866411 | Val Acc: 0.499478 loss: 15.856680
Model Saved!
[008/030] 27.57 sec(s) Train Acc: 0.520590 Loss: 255.472964 | Val Acc: 0.529432 loss: 14.766501
Model Saved!
[009/030] 28.09 sec(s) Train Acc: 0.542167 Loss: 243.480652 | Val Acc: 0.516545 loss: 15.190772
[010/030] 26.91 sec(s) Train Acc: 0.562814 Lo

In [14]:
x_test = []
raw_test = np.genfromtxt('test.csv', delimiter=',', dtype=str, skip_header=1)
for i in range(len(raw_test)):
    image = np.array(raw_test[i, 1].split(' ')).reshape(1, 48, 48)
    x_test.append(image)
x_test = np.array(x_test, dtype=float) / 255.0
x_test = torch.FloatTensor(x_test)
print(x_test.size())

torch.Size([7178, 1, 48, 48])


In [28]:
with torch.no_grad():
    test_pred = model(x_test.to(device))
    value, indices = torch.max(test_pred, dim = 1)
    print(indices)

tensor([3, 6, 3,  ..., 3, 0, 2], device='mps:0')


In [29]:
indices = indices.cpu().numpy()

df = pd.DataFrame({'id': range(len(indices)), 'label': indices})

df.to_csv('output.csv', index=False)